In [1]:
import numpy as np
from sklearn.metrics import pairwise_distances_chunked

In [19]:
# Without reduce_func:

X = np.random.RandomState(0).rand(5, 3)
print(X)

[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]
 [0.43758721 0.891773   0.96366276]
 [0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606]]


In [20]:
D_chunk = next(pairwise_distances_chunked(X))
D_chunk

array([[0.        , 0.29473397, 0.41689499, 0.19662693, 0.57216693],
       [0.29473397, 0.        , 0.57586803, 0.41860234, 0.76350759],
       [0.41689499, 0.57586803, 0.        , 0.44940452, 0.90274337],
       [0.19662693, 0.41860234, 0.44940452, 0.        , 0.51150232],
       [0.57216693, 0.76350759, 0.90274337, 0.51150232, 0.        ]])

In [22]:
# Retrieve all neighbors and average distance within radius r:

r = .2
def reduce_func(D_chunk, start):
    neigh = [np.flatnonzero(d < r) for d in D_chunk]
    avg_dist = (D_chunk * (D_chunk < r)).mean(axis=1)
    ret = 10
    return neigh, avg_dist

gen = pairwise_distances_chunked(X, reduce_func=reduce_func)
neigh, avg_dist = next(gen)
neigh

avg_dist

array([0.03932539, 0.        , 0.        , 0.03932539, 0.        ])

In [5]:
# Where r is defined per sample, we need to make use of start:

r = [.2, .4, .4, .3, .1]
def reduce_func(D_chunk, start):
    neigh = [np.flatnonzero(d < r[i])
             for i, d in enumerate(D_chunk, start)]
    return neigh

neigh = next(pairwise_distances_chunked(X, reduce_func=reduce_func))
neigh

[array([0], dtype=int64),
 array([1], dtype=int64),
 array([2], dtype=int64),
 array([3], dtype=int64),
 array([4], dtype=int64)]

In [6]:
# Force row-by-row generation by reducing working_memory:

gen = pairwise_distances_chunked(X, reduce_func=reduce_func,
                                 working_memory=0)
next(gen)

C:\Users\u004637\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\__init__.py:603: UserWarning: Could not adhere to working_memory config. Currently 0MiB, 1MiB required.
  (working_memory, np.ceil(row_bytes * 2 ** -20)))


[array([0], dtype=int64)]

In [7]:
next(gen)

[array([1], dtype=int64)]